In [2]:
#!pip install cdsapi

In [1]:
import cdsapi
import os
from zipfile import ZipFile
import glob
import boto3
import shutil
import argparse
import logging
import yaml
from yaml.loader import SafeLoader
logging.basicConfig(format='%(asctime)s %(message)s')


In [8]:
ymlPath = 'conf/chirps.yml'


key="187933:64547c55-b131-4cec-a39e-f455bf2b12b3"
url = "https://cds.climate.copernicus.eu/api/v2"
user = "129570"
bucket = "climate-action-datalake"
localDownloadPath = "data/"
zipName="sample.zip"
datalakeZone="landing"
source="agera5"
# print(key)


s3_client = boto3.client('s3')

#variableNames = ["2m_relative_humidity"] # 2m_relative_humidity-solar_radiation_flux-10m_wind_speed
days=range(1,32)
#variableS3= "2m_relative_humidity" #  2m_relative_humidity-10m-wind-speed-solar-radiation-flux


In [ ]:
def readYamlConf(path: str):
    with open(path,'r') as conf:
        confArgs = yaml.load(conf, Loader= SafeLoader)
        #print(confArgs)
    return confArgs

def downloadAgera5Data(variableName:str
                       ,statistics:[]
                       ,year:[]
                       ,month:[]
                       ,day:[]
                       ,bucket:str
                       ,downloadPath:str
                       ,zipName:str
                       ,datalakeZone:str
                       ,source:str
                       ,variable:str
                       ,url:str
                       ,key:str
                      ):
    cds =cdsapi.Client(key=key,url=url)
    sisK="sis-agrometeorological-indicators"
    for _year in year:
        for _month in month:
            for _day in day:
                jsonCopernicus = {'format':'zip'
                                   ,'version': '1_1'
                            #  ,'statistic':statistics
                                ,'variable': variableName
                                ,'year':[str(_year)]
                                ,'month':[f"{_month:02d}"]
                                ,'day':[f"{_day:02d}"]  
                                ,'time': '12_00'
                              }
                try:

                    cds.retrieve(sisK,jsonCopernicus,localDownloadPath+zipName)
                    with  ZipFile(downloadPath+zipName,'r') as z:
                        z.extractall(path=downloadPath)
                    filesUpload = glob.glob(downloadPath+"*.nc")
                    for _file in filesUpload:
                        s3Path= f"zone={datalakeZone}/source={source}/variable={variable}/"
                        response = s3_client.upload_file(_file
                                                         , bucket
                                                         ,s3Path+_file.split("/")[1])
                        os.remove(_file)
                        
                except:
                    print(_year,_month,_day)

def callDownloadData(arguments):
    if arguments.year and arguments.month and arguments.variable:
        downloadAgera5Data(variableName=variableNames
                   ,statistics=statistics
                   ,year=arguments.year
                   ,month=arguments.month
                   ,day=days
                   ,bucket=bucket
                   ,downloadPath=localDownloadPath
                   ,zipName=zipName
                   ,datalakeZone=datalakeZone
                   ,source=source
                   ,variable=arguments.variable
                   ,url=url
                   ,key=key
                  )
    else:
        logging.info(msg:"Nothing arguments passes")
        

In [ ]:
cds =cdsapi.Client(key=key,url=url)

In [9]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--year')
    parser.add_argument('--month')
    parser.add_argument('--variable')
    args = parser.parse_args()
    logging.info("Starting download rutine")
    callDownloadData(args)
    

In [ ]:
# os.remove(filesUpload[1])

In [4]:
import glob
filesUpload = glob.glob("/home/ec2-user/SageMaker/datalake/data/agera/solarFlux/brozen/"+"*.nc")

In [5]:
filesUpload

['/home/ec2-user/SageMaker/datalake/data/agera/solarFlux/brozen/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240102_final-v1.1.nc',
 '/home/ec2-user/SageMaker/datalake/data/agera/solarFlux/brozen/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240103_final-v1.1.nc',
 '/home/ec2-user/SageMaker/datalake/data/agera/solarFlux/brozen/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240104_final-v1.1.nc',
 '/home/ec2-user/SageMaker/datalake/data/agera/solarFlux/brozen/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240101_final-v1.1.nc']